# Skip-Gram+NGE数据处理模块

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



**目录：**
1. 词表构建

2. 正样本生成

3. 负样本采样

4. 负样本生成

---

In [26]:
import numpy as np
from collections import deque

In [27]:
input_file = open("/content/drive/MyDrive/Colab Notebooks/word2vec_notebook/data/text8.txt","r",encoding="utf-8")

In [28]:
def _init_dict(min_count):
    word_count_sum = 0 #统计word的数量
    sentence_count = 0 #统计句子多少个
    word2id_dict = dict() 
    id2word_dict = dict()
    wordid_frequency_dict = dict() #word_id的频率，key是id，value是频率
    word_freq = dict()
    for line in input_file:
        line = line.strip().split() #对于句子用空格来划分
        word_count_sum +=len(line)
        sentence_count +=1
        for i,word in enumerate(line):
            if i%1000000==0: # 每便利一百万个词就输出一下
                print (i,len(line))
            if word_freq.get(word)==None: # 如果不在就为1，如果在的话就+1
                word_freq[word] = 1
            else:
                word_freq[word] += 1
    for i,word in enumerate(word_freq):
        if i % 100000 == 0:
            print(i, len(word_freq))
            # 如果小于min_count就将他视为比较稀有的词，忽略
        if word_freq[word]<min_count: #给定min_count这个超参数，如果小于这个就将他看成是一个比较稀有的词
            # 如果这个词比较稀有就不要了
            # 如果不要了的话，总的词频减去这个词的词频
            word_count_sum -= word_freq[word]
            continue
        word2id_dict[word] = len(word2id_dict) # 按照顺序下去，第几个进入的词分配第几
        id2word_dict[len(id2word_dict)] = word # 同样是按照顺序排下去
        wordid_frequency_dict[len(word2id_dict)-1] = word_freq[word]
    word_count =len(word2id_dict) # 去重之后词数量
    return word2id_dict,id2word_dict,wordid_frequency_dict

In [29]:
word2id_dict,id2word_dict,wordid_frequency_dict = _init_dict(3)

0 17005207
1000000 17005207
2000000 17005207
3000000 17005207
4000000 17005207
5000000 17005207
6000000 17005207
7000000 17005207
8000000 17005207
9000000 17005207
10000000 17005207
11000000 17005207
12000000 17005207
13000000 17005207
14000000 17005207
15000000 17005207
16000000 17005207
17000000 17005207
0 253854
100000 253854
200000 253854


In [30]:
word2id_dict

{'anarchism': 0,
 'originated': 1,
 'as': 2,
 'a': 3,
 'term': 4,
 'of': 5,
 'abuse': 6,
 'first': 7,
 'used': 8,
 'against': 9,
 'early': 10,
 'working': 11,
 'class': 12,
 'radicals': 13,
 'including': 14,
 'the': 15,
 'diggers': 16,
 'english': 17,
 'revolution': 18,
 'and': 19,
 'sans': 20,
 'culottes': 21,
 'french': 22,
 'whilst': 23,
 'is': 24,
 'still': 25,
 'in': 26,
 'pejorative': 27,
 'way': 28,
 'to': 29,
 'describe': 30,
 'any': 31,
 'act': 32,
 'that': 33,
 'violent': 34,
 'means': 35,
 'destroy': 36,
 'organization': 37,
 'society': 38,
 'it': 39,
 'has': 40,
 'also': 41,
 'been': 42,
 'taken': 43,
 'up': 44,
 'positive': 45,
 'label': 46,
 'by': 47,
 'self': 48,
 'defined': 49,
 'anarchists': 50,
 'word': 51,
 'derived': 52,
 'from': 53,
 'greek': 54,
 'without': 55,
 'archons': 56,
 'ruler': 57,
 'chief': 58,
 'king': 59,
 'political': 60,
 'philosophy': 61,
 'belief': 62,
 'rulers': 63,
 'are': 64,
 'unnecessary': 65,
 'should': 66,
 'be': 67,
 'abolished': 68,
 'alth

In [31]:
# 把语句中的词转化为数字表示
def get_wordId_list():
    input_file = open("/content/drive/MyDrive/Colab Notebooks/word2vec_notebook/data/text8.txt", encoding="utf-8")
    sentence = input_file.readline()
    wordId_list = []  # 一句中的所有word 对应的 id
    sentence = sentence.strip().split(' ')
    for i,word in enumerate(sentence):
        if i%1000000==0:
            print (i,len(sentence))
        try:
            word_id = word2id_dict[word]
            wordId_list.append(word_id)
        except:
            continue
    return wordId_list

In [32]:
wordId_list = get_wordId_list()

0 17005207
1000000 17005207
2000000 17005207
3000000 17005207
4000000 17005207
5000000 17005207
6000000 17005207
7000000 17005207
8000000 17005207
9000000 17005207
10000000 17005207
11000000 17005207
12000000 17005207
13000000 17005207
14000000 17005207
15000000 17005207
16000000 17005207
17000000 17005207


In [33]:
wordId_list[:5]

[0, 1, 2, 3, 4]

In [34]:
# 生成正样本
# window_size左右多远的距离表示周围词
# index表示现在读取到哪儿
# word_pairs_queue表示队列

def get_batch_pairs(batch_size,window_size,index,word_pairs_queue):
    while len(word_pairs_queue) < batch_size:
        if index==len(wordId_list):
            index = 0
        for _ in range(1000):
            for i in range(max(index - window_size, 0),min(index + window_size + 1,len(wordId_list))):
                wordId_w = wordId_list[index] # 中心词
                wordId_v = wordId_list[i] # 周围词
                if index == i:  # 上下文=中心词 跳过
                    continue
                word_pairs_queue.append((wordId_w, wordId_v))
            index+=1 # 正样本加一
    result_pairs = []  # 返回mini-batch大小的正采样对
    for _ in range(batch_size):
        result_pairs.append(word_pairs_queue.popleft()) #每次出batch个
    return result_pairs

In [35]:
index = 0
word_pairs_queue = deque()
result_pairs = get_batch_pairs(32,3,index,word_pairs_queue)

In [36]:
result_pairs

[(0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 0),
 (2, 1),
 (2, 3),
 (2, 4),
 (2, 5),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 4),
 (3, 5),
 (3, 6),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 5),
 (4, 6),
 (4, 7),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 6),
 (5, 7),
 (5, 8),
 (6, 3),
 (6, 4)]

In [37]:
# 负采样词表
def _init_sample_table():
    sample_table = []
    sample_table_size = 1e8
    pow_frequency = np.array(list(wordid_frequency_dict.values())) ** 0.75 # 做0.75次方，减少频率大的词的抽样概率
    word_pow_sum = sum(pow_frequency) #归一化的参数
    ratio_array = pow_frequency / word_pow_sum #计算出词出现的概率
    word_count_list = np.round(ratio_array * sample_table_size) # 出现的概率乘词表的大小，然后得到其出现概率的估计
    for word_index, word_freq in enumerate(word_count_list):
        sample_table += [word_index] * int(word_freq) #按照频率将每个词加到词表里面去，比如‘我’应该出现10次，就是
                                  #['我']*10
    sample_table = np.array(sample_table)
    return sample_table

In [38]:
[10]*10

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

In [39]:
sample_table = _init_sample_table()

In [41]:
sample_table

array([     0,      0,      0, ..., 100037, 100037, 100037])

In [42]:
#开始负采样
def get_negative_sampling(positive_pairs, neg_count):
    neg_v = np.random.choice(sample_table, size=(len(positive_pairs), neg_count)).tolist()
    return neg_v

In [43]:
neg_v = get_negative_sampling(result_pairs,3)

In [44]:
neg_v

[[22280, 75967, 3],
 [6844, 25350, 3],
 [88, 19992, 1195],
 [11154, 45709, 12173],
 [3662, 4185, 6260],
 [10536, 10790, 1649],
 [2036, 265, 71309],
 [341, 3275, 21977],
 [29, 7752, 29168],
 [3756, 5, 45467],
 [1146, 89, 26],
 [90825, 2828, 24561],
 [308, 379, 14336],
 [4642, 396, 3438],
 [581, 34791, 37438],
 [1368, 16660, 44838],
 [13788, 51298, 77297],
 [51271, 66136, 14592],
 [74, 251, 114],
 [68310, 33561, 6402],
 [67616, 2839, 92716],
 [4810, 950, 208],
 [15633, 9655, 71132],
 [16147, 20780, 9276],
 [1921, 1589, 1585],
 [413, 79937, 33],
 [8375, 4972, 3310],
 [197, 18565, 11342],
 [1032, 64, 77883],
 [10765, 7549, 7928],
 [16604, 77939, 123],
 [42260, 1414, 1555]]